In [2]:
import sys

# Add the directory of promptbench to the Python path
sys.path.append('/Users/iwatson/Repos/promptbench')

# Now you can import promptbench by name
import promptbench as pb

/Users/iwatson/Documents/Research Project/prompt-optimisation/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# print all supported datasets in promptbench
print('All supported datasets: ')
for dataset in pb.SUPPORTED_DATASETS:
    print(f'  {dataset}')

All supported datasets: 
  sst2
  cola
  qqp
  mnli
  mnli_matched
  mnli_mismatched
  qnli
  wnli
  rte
  mrpc
  mmlu
  squad_v2
  un_multi
  iwslt2017
  math
  bool_logic
  valid_parentheses
  gsm8k
  csqa
  bigbench_date
  bigbench_object_tracking
  last_letter_concat
  numersense
  qasc
  bbh
  drop
  arc-easy
  arc-challenge


In [4]:
# print all supported models in promptbench
print('All supported models: ')
for model in pb.SUPPORTED_MODELS:
    print(f'  {model}')

All supported models: 
  google/flan-t5-large
  llama2-7b
  llama2-7b-chat
  llama2-13b
  llama2-13b-chat
  llama2-70b
  llama2-70b-chat
  phi-1.5
  phi-2
  palm
  gpt-3.5-turbo
  gpt-4
  gpt-4-1106-preview
  gpt-3.5-turbo-1106
  gpt-4-0125-preview
  gpt-3.5-turbo-0125
  gpt-4-turbo
  gpt-4o
  vicuna-7b
  vicuna-13b
  vicuna-13b-v1.3
  google/flan-ul2
  gemini-pro
  mistralai/Mistral-7B-v0.1
  mistralai/Mistral-7B-Instruct-v0.1
  mistralai/Mixtral-8x7B-v0.1
  mistralai/Mixtral-8x7B-Instruct-v0.1
  01-ai/Yi-6B
  01-ai/Yi-34B
  01-ai/Yi-6B-Chat
  01-ai/Yi-34B-Chat
  baichuan-inc/Baichuan2-7B-Base
  baichuan-inc/Baichuan2-13B-Base
  baichuan-inc/Baichuan2-7B-Chat
  baichuan-inc/Baichuan2-13B-Chat


In [5]:
dataset = pb.DatasetLoader.load_dataset("gsm8k")
dataset[:5]

[{'content': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
  'label': '18'},
 {'content': 'A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?',
  'label': '3'},
 {'content': 'Josh decides to try flipping a house.  He buys a house for $80,000 and then puts in $50,000 in repairs.  This increased the value of the house by 150%.  How much profit did he make?',
  'label': '70000'},
 {'content': 'James decides to run 3 sprints 3 times a week.  He runs 60 meters each sprint.  How many total meters does he run a week?',
  'label': '540'},
 {'content': "Every day, Wendi feeds each of her chickens three cups of mixed chicken feed, containing seeds, mealworms and vegetables to help keep them healthy.  She giv

In [6]:
model = "gpt-4o"

In [7]:
model = pb.LLMModel(model=model, max_new_tokens=10, temperature=0.0)

In [8]:
prompts = pb.Prompt([
    'Solve the following maths problem: {content}',
    'Solve the following elementary arithmetic problem: {content}. Provide only the final answer as a numerical value and ensure it is formatted correctly.',
    'Solve the following grade school math problem using only elementary arithmetic operations (addition, subtraction, multiplication, and division). Provide the answer as a single numerical value: {content}',
    'Solve the following math problem using only addition, subtraction, multiplication, division, and parentheses: {content}. Negative numbers and decimals are permitted. Provide the answer as a single decimal number rounded to two decimal places. Do not show intermediate steps.'
])

In [9]:
def proj_func(pred):
    mapping = {
        "correct": 1,
        "incorrect": 0
    }
    return mapping.get(pred, -1)

In [11]:
from tqdm import tqdm
for prompt in prompts:
    preds = []
    labels = []
    for data in tqdm(dataset[:100]):
        # process input
        input_text = pb.InputProcess.basic_format(prompt, data)
        label = data['label']
        raw_pred = model(input_text)
        # process output
        pred = pb.OutputProcess.cls(raw_pred)
        preds.append(pred)
        labels.append(label)
    
    # evaluate
    score = pb.Eval.compute_cls_accuracy(preds, labels)
    print(f"{score:.3f}, {prompt}")

100%|██████████| 100/100 [01:06<00:00,  1.51it/s]


0.000, Solve the following maths problem: {content}


100%|██████████| 100/100 [00:56<00:00,  1.76it/s]


0.520, Solve the following elementary arithmetic problem: {content}. Provide only the final answer as a numerical value and ensure it is formatted correctly.


100%|██████████| 100/100 [01:13<00:00,  1.35it/s]


0.000, Solve the following grade school math problem using only elementary arithmetic operations (addition, subtraction, multiplication, and division). Provide the answer as a single numerical value: {content}


100%|██████████| 100/100 [01:10<00:00,  1.42it/s]

0.020, Solve the following math problem using only addition, subtraction, multiplication, division, and parentheses: {content}. Negative numbers and decimals are permitted. Provide the answer as a single decimal number rounded to two decimal places. Do not show intermediate steps.
